In [17]:
import math
import numpy as np
import pandas as pd
from decimal import Decimal
import matplotlib.pyplot as plt

In [262]:
def f(x,y):
    return 418.9829*2 - x*np.sin(np.sqrt(abs(x))) - y*np.sin(np.sqrt(abs(y)))

In [50]:
f.__code__.co_argcount

2

### Estructura cromosómica

In [15]:
class Gen:
    def __init__(self,dominio,precision):
        """
        Parameters
        ----------
        dominio : tuple
            Límites (liminf, limsup) de la variable en cuestión.
        precision : int
            Número de cifras decimales luego del punto para la variable en cuestión. Debe ser mayor al máximo número de decimales en el dominio
        """       
        self.dominio = dominio
        self.precision = precision
        self.len = self._length()

        # Offset lb -> all(0) ; ub -> all(1)
        self.norm = (Decimal(str(dominio[1])) - Decimal(str(dominio[0]))) *10**precision / int('1'*self.len,2)
        self.offset = Decimal(str(dominio[0])) * 10**precision

    
    def _length(self):
        """Calcula el número de bits en un gen.

        Returns
        -------
        int
            Número de bits del gen <-> variable.
        """
        # Limites
        rango = self.dominio[1] - self.dominio[0]
        cell_count = math.ceil(np.log2(rango))

        # Decimales
        cell_count += math.ceil(np.log2(10**self.precision - 1))
        return cell_count
    
    def translate(self,x,mode='decode'):
        '''
        Traductor para codificar/decodificar (reales <-> binario) genes. 

        Parameters
        ----------
        x : float or str
            Gen a traducir (float si es un real, str si es binario).
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str or float
            Gen traducido (str si x era un real, float si x era binario).
        '''
        
        if mode == 'code':
            ## Real -> Binario
            
            # Cortar decimales por redondeo
            x = float(x) # failsafe
            x = round(x,self.precision)

            # Eliminar punto
            x_int,x_dec = str(x).split(".")
            x_dec = x_dec.ljust(self.precision,"0") # fill precision

            # Representación entera
            x_rep = int(x_int + x_dec)

            # Offset -> ran >= 0
            x_rep = (Decimal(str(x_rep)) - self.offset) / self.norm
            x_rep = round(x_rep) # Nuevos decimales son de orden mayor a la precision
            return np.binary_repr(x_rep, width=self.len)
        
        elif mode == 'decode':
            ## Binario -> Real

            # Convierte a entero base 10
            x = int(x,2)

            # Extraer Offset y Normalizar
            x = x * self.norm + self.offset
            x = str(round(x)) # Nuevos decimales son de orden mayor a la precision

            # Añadir el punto decimal
            int_part = x[:-self.precision]
            dec_part = x[-self.precision:]
            x_float = Decimal(int_part + '.' + dec_part)

            return float(x_float)

In [13]:
class Cromosoma:
    def __init__(self,*genes):
        """
        Parameters
        ----------
        genes : Gen
            Gen(es) del cromosoma ordenado(s)
        """       
        self.genes = genes
        self.len = sum([gen.len for gen in genes])

    def representacion(self,x,mode):
        '''
        Traductor para codificar/decodificar (reales <-> binario) cromosomas a través de sus genes.

        Parameters
        ----------
        x : list o str
            Expresión del cromosoma (lista si los genes se expresan en números reales, str si el cromosoma es binario)
        mode : str (code,decode)
            Uno de dos modos: 'code' para pasar de número real a binario, y 'decode' para pasar de binario a real.

        Returns
        -------
        str o list
            Cromosoma traducido (elementos tipo str si "genes" era una lista, lista si "genes" era binario).
        '''
        
        # liminf = self.dominio[0]
        # liminf = Decimal(str(liminf))

        if mode == 'code':
            ## Real -> Binario
            genes_bin = [gen.translate(val, mode=mode) for val,gen in zip(x,self.genes)]
            self.cromosoma = ''.join(genes_bin)
            return self.cromosoma
        
        elif mode == 'decode':
            ## Binario -> Real
            genes_lens = [gen.len for gen in self.genes]
            ind_final = np.cumsum(genes_lens)
            genes_r = [gen.translate(x[ind-lens:ind]) for gen, lens, ind in zip(self.genes, genes_lens, ind_final)]
            return genes_r

### Generación aleatoria

In [12]:
def foo(*kwargs):
    print(kwargs)
    # print(list(kwargs.values()))

foo(1,2,3,4)
# foo(x=1,y=2)


(1, 2, 3, 4)


In [210]:
liminf = -21.4574
reaal = -20.0245
Decimal(str(reaal)) + Decimal(str(abs(liminf)))

Decimal('1.4329')

In [162]:
liminf = -21.4574*10**4
reaal = -20.0245 *10**4
reaal + abs(liminf)

14329.0

In [18]:
# arbitrary number of genes
Gen1 = Gen((-500,500),3)
Gen2 = Gen((-500,500),5)
CromA = Cromosoma(Gen1,Gen2)

In [231]:
Gen1 = Gen((-500,500),3)
Gen2 = Gen((-500,500),5)
CromA = Cromosoma(genes=(Gen1,Gen2))

In [19]:
N = 30
crom_len = CromA.len
pp = np.random.binomial(n=1, p=0.5, size=(N,crom_len))

pp = pp.astype(str)
pp_bin = np.array([''.join(xi) for xi in pp])

In [20]:
_realrep = np.array([CromA.representacion(cr,mode='decode') for cr in pp_bin])
__binrep = np.array([CromA.representacion(cr,mode='code') for cr in _realrep])

for i,j in zip(pp_bin,__binrep):
    if i!=j:
        print('i: ', CromA.representacion(i,mode='decode'), i)
        print('j: ', CromA.representacion(j,mode='decode'), j)
        print('#####################')

i:  [-418.765, 239.40144] 00010100110010111101101111010100100101101001100
j:  [-418.765, 239.40144] 00010100110010111101101111010100100101101001101
#####################
i:  [4.724, -249.64008] 10000001001101011001010000000001011110010110100
j:  [4.724, -249.64008] 10000001001101011001010000000001011110010110011
#####################
i:  [-33.529, -250.79232] 01110111011010101010001111111100110000010011001
j:  [-33.529, -250.79232] 01110111011010101010001111111100110000010011000
#####################
i:  [-456.008, -324.6111] 00001011010000110001001011001110011001001001100
j:  [-456.008, -324.6111] 00001011010000110001001011001110011001001001011
#####################
i:  [-395.206, 270.26561] 00011010110100111100110001010011000000100000100
j:  [-395.206, 270.26561] 00011010110100111100110001010011000000100000011
#####################
i:  [169.29, 141.10996] 10101011010101101001101001000001111111001000001
j:  [169.29, 141.10996] 10101011010101101001101001000001111111001000010
##########

In [21]:
_realrep
   

array([[-418.765  ,  239.40144],
       [   4.724  , -249.64008],
       [ 322.98   ,  225.7134 ],
       [ 413.189  ,  -83.59369],
       [ -90.992  , -401.50003],
       [ -33.529  , -250.79232],
       [ 211.15   ,  369.5504 ],
       [-418.128  ,  441.36653],
       [ 465.819  ,  367.98943],
       [-452.341  , -148.88363],
       [ 416.194  ,  213.1971 ],
       [-300.21   ,  -30.52776],
       [-210.928  , -197.70191],
       [-456.008  , -324.6111 ],
       [-251.8    , -357.38771],
       [ 107.522  , -172.14284],
       [-425.829  ,  -43.60397],
       [-395.206  ,  270.26561],
       [ 169.29   ,  141.10996],
       [-404.475  ,   77.68261],
       [-168.25   ,  303.28398],
       [  -5.466  , -481.4669 ],
       [-149.489  ,  243.40278],
       [-279.72   , -124.2217 ],
       [ 341.711  , -369.22052],
       [-382.11   , -399.54816],
       [-225.209  ,  410.94005],
       [  33.177  , -250.26148],
       [-163.711  ,  213.25166],
       [  86.27   ,  187.95249]])

In [56]:
class SimpleEvolutionary:
    # Constructor
    def __init__(self,f,dominios,precisiones):
        """
        Parameters
        ----------
        f : function
            Función a optimizar por el algoritmo.
        dominios : tuple, list
            Iterable que contiene tuplas (liminf, limsup) de los dominios de las variables para la función "f".
        precisiones : tuple, list
            Iterable que contiene el número (int) de cifras decimales luego del punto para cada variable.
        """
        self.f = f
        self.dominios = dominios
        self.precisiones = precisiones

        # Lóngitudes de genes y cromosoma
        self.gen_lens = [self.gen_length(dom,prc) for dom,prc in zip(dominios,precisiones)]
        self.chrom_len = sum(self.gen_lens)
        
    def random_generator(N,size):
        '''
        Rutina de generación aleatoria de individuos

        Parameters
        ----------
        N : int
            Número de individuos en la población.
        size : int
            Longitud del cromosoma característico de los individuos en la población.

        Returns
        -------
        tuple (array,array)
            Array de cromosomas generados y un array con la aptitud de cada individuo.
        '''
        # Creación de población
        poblacion = np.random.binomial(n=1, p=0.5, size=(N,size)).astype(str)
        poblacion = np.array([''.join(individuo) for individuo in poblacion]) 

        # Cálculo de aptitud
        apts = np.array([f(*CromA.representacion(individuo,mode='decode')) for individuo in poblacion])

        return poblacion, apts
    
    ########################
    #### REPRESENTACIÓN ####


In [102]:
x = 486.1345678
str(Decimal(str(x)))

'486.1345678'

In [103]:
x = -486.1345678
x = Decimal(str(x))
print(x)
liminf = -500
liminf = Decimal(str(liminf))
print(liminf)
# Signo -> trasladar a rango>=0
x += abs(liminf)
print(x)
# Eliminar punto
x_int = int(str(x).replace(".",""))
print(x_int)
np.binary_repr(x_int)

-486.1345678
-500
13.8654322
138654322


'1000010000111011001001110010'

In [92]:
int(np.binary_repr(x_int),2)

98613478

In [87]:

presicion = 3

x = str(int(x,2))


# Añadir el punto decimal
int_part = x[:-self.precision[locus]]
dec_part = x[self.precision[locus]:]
return float(int_part + '.' + dec_part)

'abcd'

In [70]:
".".join(('312','009'))

'312.009'

In [94]:
## Real -> Binario
x = -421.849878
x = Decimal(str(x))
print(x)
liminf = -500
liminf = Decimal(str(liminf))
print(liminf)
# Signo -> trasladar a rango>=0
x += abs(liminf)
print(x)
# Eliminar punto
x_int = str(x).replace(".","")
print(x_int)
# np.binary_repr(x_int, width=)

-421.849878
-500
78.150122
78150122


### Representación: Transformando reales



### Población: Generación aleatoria

In [5]:
def aptitud(binary):
    '''
    Traduce las entradas binarias a entradas reales para ser evaluadas en la función objetivo "f".
    '''
